## Ch `07`: Concept `01`

## Autoencoder

All we'll need is TensorFlow and NumPy:

In [1]:
import tensorflow as tf
import numpy as np

Instead of feeding all the training data to the training op, we will feed data in small batches:

In [2]:
def get_batch(X, size):
    a = np.random.choice(len(X), size, replace=False)
    return X[a]

In [3]:
data = np.array(range(1,101))
print(get_batch(data, 10))

[97 78 89 49 15 61 56 73 80 42]


Define the autoencoder class:

In [4]:
class Autoencoder:
    # Initialize variables
    def __init__(self, input_dim, hidden_dim, epoch=500, batch_size=10, learning_rate=0.005):
        self.epoch = epoch #Number of learning cycles
        self.batch_size = batch_size
        self.learning_rate = learning_rate # Hyper-parameter of optimizer

        # Define input placeholder
        x = tf.placeholder(dtype=tf.float32, shape=[None, input_dim])
        
        # Define variables
        # 用name_scope而不用擔心變數相同
        with tf.name_scope('encode'):
            weights = tf.Variable(tf.random_normal([input_dim, hidden_dim], dtype=tf.float32), name='weights')
            biases = tf.Variable(tf.zeros([hidden_dim]), name='biases')
            encoded = tf.nn.sigmoid(tf.matmul(x, weights) + biases)
        with tf.name_scope('decode'):
            weights = tf.Variable(tf.random_normal([hidden_dim, input_dim], dtype=tf.float32), name='weights')
            biases = tf.Variable(tf.zeros([input_dim]), name='biases')
            decoded = tf.matmul(encoded, weights) + biases
        
        # These will be method variables
        self.x = x
        self.encoded = encoded
        self.decoded = decoded

        # Define cost function and training op
        # Define the reconstruction cost
        self.loss = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(self.x, self.decoded))))
        self.all_loss = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(self.x, self.decoded)), 1))
        
        # Choose the optimizer
        self.train_op = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss)
        
        # Define a saver op
        # Setup a saver to save model parameters as they’re being learned
        self.saver = tf.train.Saver()
        
    # Train on a dataset
    def train(self, data):
        # ❶ Start a TensorFlow session and initialize all variables
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            # ❷ Iterate through the number of cycles defined in the constructor
            writer = tf.summary.FileWriter("Autoencoder/", graph = sess.graph)
            for i in range(self.epoch):
                 # ❸ One-by-one train the neural network on a data item
                for j in range(500):
                    batch_data = get_batch(data, self.batch_size)
                    l, _ = sess.run([self.loss, self.train_op], feed_dict={self.x: batch_data})
                # ❹ Print the reconstruction error once every 50 cycles
                if i % 50 == 0:
                    print('epoch {0}: loss = {1}'.format(i, l))
                    # ❺ Save the learned parameters to file
                    self.saver.save(sess, './model.ckpt')
            self.saver.save(sess, './model.ckpt')
    
    # Test on some new data
    def test(self, data):
        with tf.Session() as sess:
            # Load the learned parameters
            self.saver.restore(sess, './model.ckpt')
            # Reconstruct the input
            hidden, reconstructed = sess.run([self.encoded, self.decoded], feed_dict={self.x: data})
        print('input', data)
        print('compressed', hidden)
        print('reconstructed', reconstructed)
        return reconstructed

    def get_params(self):
        with tf.Session() as sess:
            self.saver.restore(sess, './model.ckpt')
            weights, biases = sess.run([self.weights1, self.biases1])
        return weights, biases

    def classify(self, data, labels):
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            self.saver.restore(sess, './model.ckpt')
            hidden, reconstructed = sess.run([self.encoded, self.decoded], feed_dict={self.x: data})
            reconstructed = reconstructed[0]
            # loss = sess.run(self.all_loss, feed_dict={self.x: data})
            print('data', np.shape(data))
            print('reconstructed', np.shape(reconstructed))
            loss = np.sqrt(np.mean(np.square(data - reconstructed), axis=1))
            print('loss', np.shape(loss))
            horse_indices = np.where(labels == 7)[0]
            not_horse_indices = np.where(labels != 7)[0]
            horse_loss = np.mean(loss[horse_indices])
            not_horse_loss = np.mean(loss[not_horse_indices])
            print('horse', horse_loss)
            print('not horse', not_horse_loss)
            return hidden[7,:]

    def decode(self, encoding):
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            self.saver.restore(sess, './model.ckpt')
            reconstructed = sess.run(self.decoded, feed_dict={self.encoded: encoding})
        img = np.reshape(reconstructed, (32, 32))
        return img


The *Iris dataset* is often used as a simple training dataset to check whether a classification algorithm is working. The sklearn library comes with it, `pip install sklearn`.

In [5]:
from sklearn import datasets
import pandas as pd
hidden_dim = 1
data = datasets.load_iris().data
df= pd.DataFrame(data)
df.head()

,0,1,2,3
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [ ]:
input_dim = len(data[0])
input_dim,hidden_dim,data[0]

(4, 1, array([ 5.1,  3.5,  1.4,  0.2]))

In [ ]:
hidden_dim = 1
data = datasets.load_iris().data
input_dim = len(data[0])
ae = Autoencoder(input_dim, hidden_dim)
ae.train(data)
ae.test([[8, 4, 6, 2]])

epoch 0: loss = 1.314310908317566
epoch 50: loss = 0.28725141286849976
epoch 100: loss = 0.2415180653333664
epoch 150: loss = 0.3283213675022125
epoch 200: loss = 0.20978042483329773
epoch 250: loss = 0.23076164722442627
epoch 300: loss = 0.3143768906593323
epoch 350: loss = 0.21983569860458374
epoch 400: loss = 0.3222525417804718
epoch 450: loss = 0.29081740975379944


Notice how we were able to compress a 4-dimensional vector into just 1 dimension and  
then decode it back into a 4-dimensional vector with some loss in data.

## Ch `07`: Concept 02

## Autoencoder with images

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import pickle
import numpy as np

Define some helper function to load and preprocess the data:

In [ ]:
def unpickle(file):
    fo = open(file, 'rb')
    dict = pickle.load(fo, encoding='latin1')
    fo.close()
    return dict

# simply averaging the red, green, and blue values.
def grayscale(a):
    return a.reshape(a.shape[0], 3, 32, 32).mean(1).reshape(a.shape[0], -1)

In [ ]:
np.array([[1,2,3],[1,2,3]]).shape[0]

Download the CIFAR-10 dataset in Python from https://www.cs.toronto.edu/~kriz/cifar.html. Then we can load the data using the following code:

In [ ]:
print(unpickle('./cifar-10-batches-py/data_batch_1'))

In [ ]:
names = unpickle('./cifar-10-batches-py/batches.meta')['label_names']
data, labels = [], []
# Loop through the 6 files
for i in range(1, 6):
    filename = './cifar-10-batches-py/data_batch_' + str(i)
    # Load the file to obtain a Python dictionary
    batch_data = unpickle(filename)
    if len(data) > 0:
        data = np.vstack((data, batch_data['data']))
        labels = np.hstack((labels, batch_data['labels']))
    else:
        data = batch_data['data']
        labels = batch_data['labels']

data = grayscale(data)
x = np.matrix(data)
y = np.array(labels)

In [ ]:
x

In [ ]:
y

Train the autoencoder on images of horses:

In [ ]:
horse_indices = np.where(y == 7)[0]
horse_x = x[horse_indices]

In [ ]:
horse_indices

In [ ]:
np.shape(horse_x)

In [ ]:
horse_x

In [ ]:
horse_x[2, :]

In [ ]:
horse_indices = np.where(y == 7)[0]
horse_x = x[horse_indices]
print(np.shape(horse_x))  # (5000, 3072)

print('Some examples of horse images we will feed to the autoencoder for training')

#This makes the figure's width 10 inches, and its height 10 inches.
plt.rcParams['figure.figsize'] = (10, 10)
num_examples = 5
for i in range(num_examples):
    horse_img = np.reshape(horse_x[i, :], (32, 32))
    #plt.subplot(nrows幾個橫排,ncols幾行，plot_number第幾個圖)
    plt.subplot(1, num_examples, i+1)
    plt.imshow(horse_img, cmap='Greys_r')
plt.show()

In [ ]:
np.shape(horse_x)[1]

In [ ]:
input_dim = np.shape(horse_x)[1]
hidden_dim = 100
ae = Autoencoder(input_dim, hidden_dim)
ae.train(horse_x)

Test the autoencoder on other images:

In [ ]:
test_data = unpickle('./cifar-10-batches-py/test_batch')
test_x = grayscale(test_data['data'])
test_labels = np.array(test_data['labels'])
encodings = ae.classify(test_x, test_labels)